In [2]:
import pandas as pd
from datetime import datetime, date, timedelta
import requests
from pytz import timezone

In [3]:
from datetime import datetime

In [4]:
def tomorrows_flight_arrivals(icao_list):

  today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
  tomorrow = (today + timedelta(days=1))

  list_for_df = []

  for icao in icao_list:
    times = [["00:00","11:59"],["12:00","23:59"]]

    for time in times:
      url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
      querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false"}
      headers = {
          'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
          'x-rapidapi-key': "ffc96c1d46msh923da3c4ad3aa34p1573f4jsn3227997df6b9"
          }
      response = requests.request("GET", url, headers=headers, params=querystring)
      flights_json = response.json()

      for flight in flights_json['arrivals']:
        flights_dict = {}
        flights_dict['arrival_icao'] = icao
        # .get() is another way of ensuring our code doesn't break
        # in the previous 2 notebooks you learnt about 'if' (cities) and 'try/except' (weather)
        # .get() works similar, it will get the text if possible, if there is no text a None value will be inserted instead
        flights_dict['flight_num'] = flight.get('number', None)
        flights_dict['arrival_time'] = flight['arrival'].get('scheduledTimeLocal', None)
        flights_dict['arrival_terminal'] = flight['arrival'].get('terminal', None)
        flights_dict['departure_city'] = flight['departure']['airport'].get('name', None)
        flights_dict['departure_icao'] = flight['departure']['airport'].get('icao', None)
        flights_dict['airline'] = flight['airline'].get('name', None)
        flights_dict['data_retrieved_on'] = datetime.now().astimezone(timezone('Europe/Berlin')).date()
        list_for_df.append(flights_dict)

  return pd.DataFrame(list_for_df)

In [5]:
#test one first
tomorrows_flight_arrivals(['EDDB']).head()

,arrival_icao,flight_num,arrival_time,arrival_terminal,departure_city,departure_icao,airline,data_retrieved_on
0,EDDB,QR 79,2023-03-15 06:50+01:00,1,Doha,OTHH,Qatar Airways,2023-03-14
1,EDDB,BT 211,2023-03-15 07:55+01:00,1,Riga,EVRA,Air Baltic,2023-03-14
2,EDDB,DL 92,2023-03-15 07:05+01:00,1,New York,KJFK,Delta Air Lines,2023-03-14
3,EDDB,EW 2002,2023-03-15 07:50+01:00,1,Stuttgart,EDDS,Eurowings,2023-03-14
4,EDDB,EW 9048,2023-03-15 07:35+01:00,1,Duesseldorf,EDDL,Eurowings,2023-03-14


In [ ]:
# add other airport later
#tomorrows_flight_arrivals(['EDDB', 'EDDH', 'EDDF', 'EDDM', 'EDDS','EDDK','EDDV']).head()

In [28]:
#convert to datetime data types
flight_df['arrival_time'] = pd.to_datetime(flight_df['arrival_time'])
flight_df['data_retrieved_on'] = pd.to_datetime(flight_df['data_retrieved_on'])

In [26]:
flight_df = tomorrows_flight_arrivals(['EDDB']).head()

In [29]:
flight_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype                               
---  ------             --------------  -----                               
 0   arrival_icao       5 non-null      object                              
 1   flight_num         5 non-null      object                              
 2   arrival_time       5 non-null      datetime64[ns, pytz.FixedOffset(60)]
 3   arrival_terminal   5 non-null      object                              
 4   departure_city     5 non-null      object                              
 5   departure_icao     4 non-null      object                              
 6   airline            5 non-null      object                              
 7   data_retrieved_on  5 non-null      datetime64[ns]                      
dtypes: datetime64[ns, pytz.FixedOffset(60)](1), datetime64[ns](1), object(6)
memory usage: 448.0+ bytes
